In [ ]:
!pip install transformers==4.40.2 datasets
!pip install accelerate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq

In [ ]:
dataset = load_dataset('json', data_files='/content/drive/My Drive/Colab/T5-jeju/data/rm_binary2.jsonl')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
model = AutoModelForSequenceClassification.from_pretrained('klue/bert-base', num_labels=2)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./drive/MyDrive/Colab/T5-jeju/data/saved_models/roberta_rm")
model = AutoModelForSequenceClassification.from_pretrained("./drive/MyDrive/Colab/T5-jeju/data/saved_models/roberta_rm", num_labels=2)

In [ ]:
from torch import nn
sent = "또 성게도 싸게 사당으네 그거 얼마나 좋은 거라게 그 몸에"
sent_tok = tokenizer(sent, return_tensors="pt")
print(sent_tok)
output = model(**sent_tok).logits
print(output)

{'input_ids': tensor([[    2,   918,  1268,  2318,  2119,  1336,  2318, 14400,  2279,  2203,
          5139,  4296,  1560,  2073, 12370,  2318,   636,  1083,  2170,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([[ 2.7335, -3.1532]], grad_fn=<AddmmBackward0>)


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
model = model.to(device)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt").to(device)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/310732 [00:00<?, ? examples/s]

In [ ]:
data = tokenized_datasets['train']

In [ ]:
training_args = TrainingArguments(
    output_dir='./results_rm',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    per_gpu_train_batch_size=32,
    per_gpu_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./drive/MyDrive/Colab/T5-jeju/data/logs/roberta_rm',
    evaluation_strategy='epoch',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


ValueError: Trainer: evaluation requires an eval_dataset.

In [ ]:
trainer.save_model("./drive/MyDrive/Colab/T5-jeju/data/saved_models/roberta_rm")

In [ ]:
torch.cuda.empty_cache()